In [6]:
import os

import numpy as np
from scipy.ndimage import imread
from scipy.spatial.distance import cdist

# Parameters
nrun = 20  # Number of classification runs
path_to_script_dir = os.path.dirname(os.path.realpath('test.ipynb'))
path_to_all_runs = os.path.join(path_to_script_dir, 'all_runs')
fname_label = 'class_labels.txt'  # Where class labels are stored for each run

# ***********************************************************************************
def classification_run(folder, f_load, f_cost, ftype='cost'):    
    # if ftype is not in the list, arise error    
    assert ftype in {'cost', 'score'}
        
    with open(os.path.join(path_to_all_runs, folder, fname_label)) as f:
        pairs = [line.split() for line in f.readlines()]        
        # pairs = [['run01/test/item01.png',...'run01/training/class16.png']]
        
    # Unzip the pairs into two sets of tuples(元組)
    # guess: split the pairs content into file set of arrays.
    test_files, train_files = zip(*pairs)

    answers_files = list(train_files)  # Copy the training file list                
    # sorted(test_files) = ['run01/test/item01.png',..'run01/test/item20.png']
    test_files = sorted(test_files)
    train_files = sorted(train_files)
    n_train = len(train_files)
    n_test = len(test_files)    
    
    # Load the images (and, if needed, extract features)
    # f = '../all_runs/run01/class_labels.txt'
    # f_load() = load_img_as_points(filename)
    train_items = [f_load(os.path.join(path_to_all_runs, f))
                   for f in train_files]
    test_items = [f_load(os.path.join(path_to_all_runs, f))
                  for f in test_files]
    
    # Compute cost matrix
    costM = np.zeros((n_test, n_train))
    for i, test_i in enumerate(test_items):
        for j, train_j in enumerate(train_items):
            costM[i, j] = f_cost(test_i, train_j)
            
    if ftype == 'cost':
        y_hats = np.argmin(costM, axis=1)
    elif ftype == 'score':
        y_hats = np.argmax(costM, axis=1)
    else:
        # This should never be reached due to the assert above
        raise ValueError('Unexpected ftype: {}'.format(ftype))   

    # compute the error rate by counting the number of correct predictions
    correct = len([1 for y_hat, answer in zip(y_hats, answers_files)
                   if train_files[y_hat] == answer])
    pcorrect = correct / float(n_test)  # Python 2.x ensure float division
    perror = 1.0 - pcorrect

    return perror * 100

# ***********************************************************************************
#      f_load : (load_img_as_points) itemA = f_load('file.png') read/process in the image file
    # Load image file and return coordinates of black pixels in the binary image
    #
    # Input
    #  filename : string, absolute path to image
    #
    # Output:
    #  D : [n x 2] rows are coordinates
    #
# My testing: print parameters only load class01.png data
#    Mytest = os.path.join(path_to_all_runs, 'run01/training/class01.png')
#    if filename == Mytest:
#        print(D)

# ***********************************************************************************
#  f_cost : (modified_hausdorf_distance) f_cost(itemA,itemB) should compute similarity between two
#           images, using output of f_load
    # Modified Hausdorff Distance
    #
    # Input
    #  itemA : [n x 2] coordinates of black pixels
    #  itemB : [m x 2] coordinates of black pixels
    #
    #  M.-P. Dubuisson, A. K. Jain (1994). A modified hausdorff distance for object matching.
    #  International Conference on Pattern Recognition, pp. 566-568.
    #
# ***********************************************************************************
def modified_hausdorf_distance(itemA, itemB):
    D = cdist(itemA, itemB)
    
    mindist_A = D.min(axis=1)
    mindist_B = D.min(axis=0)

    mean_A = np.mean(mindist_A)
    mean_B = np.mean(mindist_B)

    return max(mean_A, mean_B)


# ***********************************************************************************
#   if filename == '/home/../one-shoting/one-shoting-demo/all_runs/run01/training/class01.png':
#       print(D - D.mean(axis=0))
# ***********************************************************************************
def load_img_as_points(filename):
    import imageio
    
    '''
    filename =
    /home/../one-shoting/one-shoting-demo/all_runs/run01/training/class01.png
    ...
    /home/../one-shoting/one-shoting-demo/all_runs/run01/test/item20.png
    '''
    #print(filename)
    
    I = imageio.imread(filename)
    #I = I.flatten()
    
    # Convert to boolean array and invert the pixel values
    I = ~np.array(I, dtype=np.bool)    
        
    # Create a new array of all the non-zero element coordinates
    # see "readme one-shot.odt" for details	
    D = np.array(I.nonzero()).T
    return D - D.mean(axis=0)


# there's 20 folder inside all_runs/	
# the following create array to hold error: e.g. perror[20]	
perror = np.zeros(nrun)

#print(load_img_as_points('class01.png'))

for r in range(nrun):
    # 'run{:02d}'.format(r + 1) --> 'run01' ... 'run20'
    # load_img_as_points and modified_hausdorf_distance are a function pointer

    perror[r] = classification_run('run{:02d}'.format(r + 1),
                                       load_img_as_points,
                                       modified_hausdorf_distance,
                                       'cost')

    
    print('run {:02d} (error {:.1f}%)'.format(r, perror[r]))

    # refer numpy_test.py
    total = np.mean(perror)
    print('Average error {:.1f}%'.format(total))

run 00 (error 45.0%)
Average error 2.2%
run 01 (error 35.0%)
Average error 4.0%
run 02 (error 40.0%)
Average error 6.0%
run 03 (error 25.0%)
Average error 7.2%
run 04 (error 30.0%)
Average error 8.8%
run 05 (error 15.0%)
Average error 9.5%
run 06 (error 60.0%)
Average error 12.5%
run 07 (error 35.0%)
Average error 14.2%
run 08 (error 40.0%)
Average error 16.2%
run 09 (error 55.0%)
Average error 19.0%
run 10 (error 15.0%)
Average error 19.8%
run 11 (error 70.0%)
Average error 23.2%
run 12 (error 65.0%)
Average error 26.5%
run 13 (error 35.0%)
Average error 28.2%
run 14 (error 15.0%)
Average error 29.0%
run 15 (error 25.0%)
Average error 30.2%
run 16 (error 30.0%)
Average error 31.8%
run 17 (error 40.0%)
Average error 33.8%
run 18 (error 70.0%)
Average error 37.2%
run 19 (error 30.0%)
Average error 38.8%
